# 魔法方法、特性和迭代器
在Python中，有些名称很特别，开头和结尾都是两个下划线。在这样的名称中，很大一部分都是魔法（特殊）方法的名称。如果你的对象实现了这些方法，它们将在特定情况下（具体是哪种情况取决于方法的名称）被Python调用，而几乎不需要直接调用。
- 请注意，在Python 3中没有旧式类，因此无需显式地继承object或将__metaclass__设置为type。所有的类都将隐式地继承object。如果没有指定超类，将直接继承它，否则将间接地继承它。

## 构造函数
构造函数不同于普通方法的地方在于，将在对象创建后自动调用它们.

In [32]:
class Foobar():
    def __init__(self, x):
        self.x = x
    
f = Foobar(42)
f.x


42

In [33]:
class Foobar():
    def __init__(self, x=None):
        self.x = x
    
f = Foobar()
print(f.x)
g = Foobar("I can be anything")
print(g.x)

None
I can be anything


- Python提供了魔法方法__del__，也称作析构函数（destructor）。这个方法在对象被销毁（作为垃圾被收集）前被调用，但鉴于你无法知道准确的调用时间，建议尽可能不要使用__del__。

### 重写普通方法和特殊的构造函数
**重写**是继承机制的一个重要方面，对构造函数来说尤其重要。构造函数用于初始化新建对象的状态，而对大多数子类来说，除超类的初始化代码外，还需要有自己的初始化代码。虽然所有方法的重写机制都相同，但与重写普通方法相比，重写构造函数时更有可能遇到一个特别的问题：**重写构造函数时，必须调用超类（继承的类）的构造函数，否则可能无法正确地初始化对象。**

In [34]:
class Bird:
    def __init__(self):
        self.hungry = True
    def eat(self):
        if self.hungry:
            print('Aaaah ...')
            self.hungry = False
        else:
            print('No, thanks!')
b = Bird()
b.eat()
b.eat()

Aaaah ...
No, thanks!


In [35]:
class SongBird(Bird):
    def __init__(self):
        self.sound = 'Squawk!'
    def sing(self):
        print(self.sound)
sb = SongBird()
sb.sing()
sb.eat()

Squawk!


AttributeError: 'SongBird' object has no attribute 'hungry'

异常清楚地指出了问题出在什么地方：SongBird没有属性hungry。为何会这样呢？因为在SongBird中重写了构造函数，但新的构造函数没有包含任何初始化属性hungry的代码。要消除这种错误，SongBird的构造函数必须调用其超类（Bird）的构造函数，以确保基本的初始化得以执行。  
为此，有两种方法：调用未关联的超类构造函数，以及使用函数super。接下来的两节将介绍这两种方法。

### 调用未关联的超类构造函数
在较新的Python版本中，显然应使用函数super（这将在下一节讨论）。然而，很多既有代码使用的都是本节介绍的方法，因此你必须对其有所了解。另外，这种方法也极具启迪意义，淋漓尽致地说明了关联方法和未关联方法之间的差别。

In [ ]:
class SongBird(Bird):
    def __init__(self):
        Bird.__init__(self)      # Call the parent constructor
        self.sound = 'Squawk!'
    def sing(self):
        print(self.sound)
sb = SongBird()
sb.sing()
sb.eat()
sb.eat()

Squawk!
Aaaah ...
No, thanks!


- 这样做为何管用呢？对实例调用方法时，方法的参数self将自动关联到实例（称为关联的方法），这样的示例你见过多个。  
- 然而，如果你通过类调用方法（如Bird.__init__），就没有实例与其相关联。在这种情况下，你可随便设置参数self。这样的方法称为未关联的。这就对本节的标题做出了解释。
- 通过将这个未关联方法的self参数设置为当前实例，将使用超类的构造函数来初始化SongBird对象。这意味着将设置其属性hungry。

### 使用函数 super
>- 如果你使用的不是旧版Python，就应使用函数super。这个函数只适用于新式类，而你无论如何都应使用新式类。  
>- 调用这个函数时，将当前类和当前实例作为参数。对其返回的对象调用方法时，调用的将是超类（而不是当前类）的方法。因此，在SongBird的构造函数中，可不使用Bird，而是使用super(SongBird, self)。另外，可像通常那样（也就是像调用关联的方法那样）调用方法__init__。在Python 3中调用函数super时，可不提供任何参数（通常也应该这样做），而它将像变魔术一样完成任务。

In [ ]:
class Bird:
    def __init__(self):
        self.hungry = True
    def eat(self):
        if self.hungry:
            print('Aaaah ...')
            self.hungry = False
        else:
            print('No, thanks!')
class SongBird(Bird):
    def __init__(self):
        super().__init__()
        self.sound = 'Squawk!'
    def sing(self):
        print(self.sound)
        
sb = SongBird()
sb.eat()
sb.eat()
sb.sing()

Aaaah ...
No, thanks!
Squawk!


- 实际上，函数super很聪明，因此即便有多个超类，也只需调用函数super一次（条件是所有超类的构造函数也使用函数super）。另外，对于使用旧式类时处理起来很棘手的问题（如两个超类从同一个类派生而来），在使用新式类和函数super时将自动得到处理。你无需知道函数super的内部工作原理，但必须知道的是，使用函数super比调用超类的未关联构造函数（或其他方法）要好得多。
- 函数super返回的到底是什么呢？通常，你无需关心这个问题，只管假定它返回你所需的超类即可。实际上，它返回的是一个super对象，这个对象将负责为你执行方法解析。当你访问它的属性时，它将在所有的超类（以及超类的超类，等等）中查找，直到找到指定的属性或引发AttributeError异常。

## 元素访问
本节将介绍一组很有用的魔法方法，让你能够创建行为类似于序列或映射的对象。我们先来解释一下什么是"协议":
>在Python中，协议通常指的是规范行为的规则，有点类似于第7章提及的接口。协议指定应实现哪些方法以及这些方法应做什么。在Python中，多态仅仅基于对象的行为（而不基于祖先，如属于哪个类或其超类等），因此这个概念很重要：其他的语言可能要求对象
属于特定的类或实现了特定的接口，而Python通常只要求对象遵循特定的协议。因此，要成为序列，只需遵循序列协议即可。

### 基本的序列和映射协议
序列和映射基本上是元素（item）的集合，要实现它们的基本行为（协议），不可变对象需要实现2个方法，而可变对象需要实现4个。

- `__len__(self)`：这个方法应返回集合包含的项数，对序列来说为元素个数，对映射来说为键-值对数。如果`__len__`返回零（且没有实现覆盖这种行为的`__nonzero__`），对象在布尔上下文中将被视为假（就像空的列表、元组、字符串和字典一样）。
- `__getitem__(self, key)`：这个方法应返回与指定键相关联的值。对序列来说，键应该是0~n-1的整数（也可以是负数，这将在后面说明），其中n为序列的长度。对映射来说，键可以是任何类型。
- `__setitem__(self, key, value)`：这个方法应以与键相关联的方式存储值，以便以后能够使用`__getitem__`来获取。当然，仅当对象可变时才需要实现这个方法。
- `__delitem__(self, key)`：这个方法在对对象的组成部分使用`__del__`语句时被调用，应删除与key相关联的值。同样，仅当对象可变（且允许其项被删除）时，才需要实现这个方法。对于这些方法，还有一些额外的要求。
- 对于序列，如果键为负整数，应从末尾往前数。换而言之，`x[-n]`应与`x[len(x)-n]`等效。
- 如果键的类型不合适（如对序列使用字符串键），可能引发`TypeError`异常。
- 对于序列，如果索引的类型是正确的，但不在允许的范围内，应引发`IndexError`异常。
要了解更复杂的接口和使用的抽象基类（Sequence），请参阅有关模块collections的文档。

In [ ]:
def check_index(key):
    """
    指定的键是否是可接受的索引？
    键必须是非负整数，才是可接受的。如果不是整数，
    将引发TypeError异常；如果是负数，将引发Index
    Error异常（因为这个序列的长度是无穷的）
    """
    if not isinstance(key, int): raise TypeError
    if key < 0: raise IndexError
class ArithmeticSequence:
    def __init__(self, start=0, step=1):
        """
        初始化这个算术序列
        start -序列中的第一个值
        step -两个相邻值的差
        changed -一个字典，包含用户修改后的值
        """
        self.start = start      # 存储起始值
        self.step = step        # 存储步长值
        self.changed = {}       # 没有任何元素被修改
    def __getitem__(self, key):
        """
        从算术序列中获取一个元素
        """
        check_index(key)
        try: return self.changed[key]               # 修改过？
        except KeyError:                            # 如果没有修改过，
            return self.start + key * self.step     # 就计算元素的值
    def __setitem__(self, key, value):
        """
        修改算术序列中的元素
        """
        check_index(key)
        self.changed[key] = value                   # 存储修改后的值

这些代码实现的是一个算术序列，其中任何两个相邻数字的差都相同。第一个值是由构造函数的参数start（默认为0）指定的，而相邻值之间的差是由参数step（默认为1）指定的。你允许用户修改某些元素，这是通过将不符合规则的值保存在字典changed中实现的。如果元素未被修改，就使用公式self.start + key * self.step来计算它的值。

In [ ]:
s = ArithmeticSequence(1, 2)
print(s[4])
s[4] = -2
print(s[4])
len(s)              # 无穷项, 我们没有给它len方法

9
-2


TypeError: object of type 'ArithmeticSequence' has no len()

In [ ]:
s[-2]       # 索引检查是由我为此编写的辅助函数check_index负责的。

IndexError: 

In [ ]:
del s[4]            # 也没有del方法

AttributeError: __delitem__

### 从 list、dict 和 str 派生
- 基本的序列/映射协议指定的4个方法能够让你走很远，但序列还有很多其他有用的魔法方法和普通方法，其中包括将在9.6节介绍的方法__iter__。要实现所有这些方法，不仅工作量大，而且难度不小。如果只想定制某种操作的行为，就没有理由去重新实现其他所有方法。这就是程序员的懒惰（也是常识）。
- 那么该如何做呢？“咒语”就是继承。在能够继承的情况下为何去重新实现呢？在标准库中，模块collections提供了抽象和具体的基类，但你也可以继承内置类型。因此，如果要实现一种行为类似于内置列表的序列类型，可直接继承list。

In [ ]:
# 一个带访问计数器的列表 #
class CounterList(list):
    def __init__(self, *args):
        super().__init__(*args)
        self.counter = 0
    def __getitem__(self, index):
        self.counter += 1
        return super(CounterList, self).__getitem__(index)

CounterList类深深地依赖于其超类（list）的行为。CounterList没有重写的方法（如append、extend、index等）都可直接使用。在两个被重写的方法中，使用super来调用超类的相应方法，并添加了必要的行为：初始化属性counter（在__init__中）和更新属性counter（在__getitem__中）。
- 注意 重写__getitem__并不能保证一定会捕捉用户的访问操作，因为还有其他访问列表内容的方式，如通过方法pop。

In [ ]:
cl = CounterList(range(10))
print(cl)
cl.reverse()
print(cl)
print(cl[1:3])
print(cl.counter)
del cl[1:3]
print(cl.counter)
print(cl)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
[8, 7]
1
1
[9, 6, 5, 4, 3, 2, 1, 0]


In [ ]:
a = list(range(10))
print(a)
print(range(10),type(range(10)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
range(0, 10) <class 'range'>


## 其他魔法方法
特殊（魔法）名称的用途很多，前面展示的只是冰山一角。魔法方法大多是为非常高级的用
途准备的，因此这里不详细介绍。然而，如果你感兴趣，可以模拟数字，让对象像函数一样被调
用，影响对象的比较方式，等等。要更详细地了解有哪些魔法方法，可参阅“Python Reference
Manual”的Special method names一节。
## 特性
第7章提到了存取方法，它们是名称类似于getHeight和setHeight的方法，用于获取或设置属
性（这些属性可能是私有的，详情请参阅7.2.4节）。如果访问给定属性时必须采取特定的措施，
那么像这样封装状态变量（属性）很重要。  


那么如何解决这个问题呢？给所有的属性都提供存取方法吗？这当然并非不可能，但如果有
大量简单的属性，这样做就不现实（而且有点傻），因为将需要编写大量这样的存取方法，除了
获取或设置属性外什么都不做。这将引入复制并粘贴（重复代码）的坏味，显然很糟糕（虽然在
有些语言中，这样的问题很常见）。所幸Python能够替你隐藏存取方法，让所有的属性看起来都
一样。通过存取方法定义的属性通常称为特性（property）。


在Python中，实际上有两种创建特定的机制，我将重点介绍较新的那种——函数property，
它只能用于新式类。随后，我将简单说明如何使用魔法方法来实现特性。

In [37]:
class Rectangle:
    def __init__(self):
        self.width = 0
        self.height = 0
    def set_size(self, size):
        self.width, self.height = size
    def get_size(self):
        return self.width, self.height
r = Rectangle()
r.width = 10
r.height = 5
r.get_size()

(10, 5)

In [38]:
r.set_size((20, 10))
r.width

20

In [39]:
class Rectangle:
    def __init__(self):
        self.width = 0
        self.height = 0
    def set_size(self, size):
        self.width, self.height = size
    def get_size(self):
        return self.width, self.height
    size = property(get_size, set_size)  # 创建属性

在这个新版的Rectangle中，通过调用函数property并将存取方法作为参数（**获取方法在前，设置方法在后**）创建了一个特性，然后将名称size关联到这个特性。这样，你就能以同样的方式对待width、height和size，而无需关心它们是如何实现的。

In [ ]:
r = Rectangle()
r.width = 10
r.height = 5
r.size

(10, 5)

In [41]:
r.size = (20, 10)
r.width

20

- 实际上，调用函数property时，还可不指定参数、指定一个参数、指定三个参数或指定四
个参数。如果没有指定任何参数，创建的特性将既不可读也不可写。如果只指定一个参数（获
取方法），创建的特性将是只读的。第三个参数是可选的，指定用于删除属性的方法（这个方
法不接受任何参数）。第四个参数也是可选的，指定一个文档字符串。这些参数分别名为fget、
fset、fdel和doc。如果你要创建一个只可写且带文档字符串的特性，可使用它们作为关键字参
数来实现。
-本节虽然很短（旨在说明函数property很简单），却非常重要。这里要说明的是，对于新式
类，应使用特性而不是存取方法。